In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
#from sklearn.datasets.samples_generator import make_blobs

!pip install lxml
!pip install bs4
from bs4 import BeautifulSoup
import requests

from geopy.geocoders import Nominatim 
import requests
! conda install -c conda-forge folium
#import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from collections import OrderedDict
import matplotlib as mpl

from pandas.io.json import json_normalize
import folium

import json

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#Foursquare Info
CLIENT_ID = 'F3AM3GHNJL0VSLIGH3F5LMF1CXB5CO43PDC2FDUNVM0DQDYN' # your Foursquare ID
CLIENT_SECRET = 'S5IPPXEXVSECSWPEAC1SBW1E00LAISVPRO2ACCNATWMVZTJX' # your Foursquare Secret
VERSION = '20210615'
Limit = 300

#other search parameters
city = 'Zürich'


#funtion to search for a query near a city, returns json file
def SearchVenues(query):
    
    search_results=[]
    for searchtopic in zip(query):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            city, 
            Limit,
            query)
        results = requests.get(url).json()['response']['groups'][0]['items']
    
        search_results.append([(
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng']) for v in results])
    
        return (search_results)

In [3]:
#creating json file with wanted content
query1 = 'restaurant'
restaurants = SearchVenues(query1)
restaurants

[[('John Baker Ltd', 47.36720835584306, 8.54729328625253),
  ('Coco Grill & Bar', 47.368958637059286, 8.538430806857162),
  ('KIN', 47.374076573465295, 8.5188527405262),
  ('Waidhof', 47.42427, 8.5251699),
  ('Kafi Dihei', 47.375239, 8.513285),
  ('169 West', 47.373884, 8.518049),
  ('John Baker', 47.376282010318235, 8.526332291424712),
  ('Café Noir', 47.38205413373607, 8.5306430877368),
  ('Samurai', 47.37374701258848, 8.527696984598784),
  ('Äss-Bar', 47.37256105232428, 8.543692839575389),
  ('Co Chin Chin', 47.38380906569936, 8.528479899712615),
  ('Restaurant Gertrudhof', 47.37379063895187, 8.515933955106151),
  ('Ooki', 47.3722774666315, 8.518805943275494),
  ('Paninoteca Il Pentagramma', 47.38112303767148, 8.533554349862975),
  ('Miki Ramen', 47.375152, 8.516653),
  ('Kokoro', 47.38068, 8.526546),
  ('Ban Song Thai', 47.369395, 8.544136),
  ('Restaurant JOSEF', 47.384005824668954, 8.52902029772919),
  ('Wirtschaft Degenried', 47.36739497739939, 8.579027236425036),
  ('Restaurant

In [4]:
query2 = 'drinks'
bars = SearchVenues(query2)
query3 = 'train station'
train = SearchVenues(query3)
query4 = 'tram station'
tram = SearchVenues(query4)
query5 = 'bus station'
bus = SearchVenues(query5)

In [6]:
#writting json file data into pd dataframe
df_rest = pd.DataFrame(item for venue_list in restaurants for item in venue_list)
df_bars = pd.DataFrame(item for venue_list in bars for item in venue_list)
df_trai = pd.DataFrame(item for venue_list in train for item in venue_list)
df_tram = pd.DataFrame(item for venue_list in tram for item in venue_list)
df_bus = pd.DataFrame(item for venue_list in bus for item in venue_list)

In [23]:
#combining all public tranpsort location to single dataframe
df_transport = df_trai.append(df_tram, ignore_index=True)
df_transport = df_transport.append(df_bus, ignore_index=True)

#edit column titles
df_rest.columns = ('Name','Lat','Lng')
df_bars.columns = ('Name','Lat','Lng')
df_transport.columns = ('Name','Lat','Lng')

df_rest.head(5)

,Name,Lat,Lng
0,John Baker Ltd,47.367208,8.547293
1,Coco Grill & Bar,47.368959,8.538431
2,KIN,47.374077,8.518853
3,Waidhof,47.424270,8.525170
4,Kafi Dihei,47.375239,8.513285


In [24]:
#view shape
print(df_rest.shape)
print(df_bars.shape)
print(df_transport.shape)

(100, 3)
(100, 3)
(273, 3)


In [32]:
#setting up k-means (Restaurants)
k_meansR = KMeans(init="k-means++", n_clusters=12, n_init=30)
k_meansR.fit(df_rest[['Lat','Lng']])
#cluster labels
RestL = k_meansR.labels_
#cluster center coordinates
RC = k_meansR.cluster_centers_

#(Bars)
k_meansB = KMeans(init="k-means++", n_clusters=12, n_init=30)
k_meansB.fit(df_bars[['Lat','Lng']])
BarL = k_meansB.labels_
BC = k_meansB.cluster_centers_

#(Transport)
k_meansT = KMeans(init="k-means++", n_clusters=12, n_init=30)
k_meansT.fit(df_transport[['Lat','Lng']])
TransL = k_meansT.labels_
TC = k_meansT.cluster_centers_

In [33]:
#append kmeans cluster number to dafa frame for easy colouring
df_rest['Cluster Nr.'] = RestL
df_bars['Cluster Nr.'] = BarL
df_transport['Cluster Nr.'] = TransL

In [73]:
#create data frames for density centers with labelled indexes
RestC = pd.DataFrame(RC)
BarC = pd.DataFrame(BC)
TransC = pd.DataFrame(TC)
RestC.columns = ('Lat','Lng')
BarC.columns = ('Lat','Lng')
TransC.columns = ('Lat','Lng')
TransC.head(5)

,Lat,Lng
0,47.413298,8.549879
1,47.366870,8.520261
2,47.379609,8.538887
3,47.396036,8.487493
4,47.358412,8.588912


In [74]:
#plotting map
address = 'Zuerich'
geolocator = Nominatim(user_agent="zuerich_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#setting cluster number as 6
k=12


def CreateMapK(df):
    # create map
    MyMap = folium.Map(location=[latitude, longitude], zoom_start=11)
    
    # setting color scheme for the clusters (R)
    X = np.arange(k)
    Ys = [i + X + (i*X)**2 for i in range(k)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(Ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # adding markers to the map (R)
    markers_colors = []
    for lat, lng, counter in zip(df['Lat'], df['Lng'], df['Cluster Nr.']):
        label = folium.Popup(str(df['Cluster Nr.']) + ' Cluster ' + str(counter), parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=rainbow[counter-1],
            fill=True,
            fill_color=rainbow[counter-1],
            fill_opacity=0.7).add_to(MyMap)
    
    return MyMap

In [75]:
#Creating map for seperate clusters
MapRest = CreateMapK(df_rest)
MapBar = CreateMapK(df_bars)
MapTrans = CreateMapK(df_transport)

In [76]:
MapRest

In [77]:
MapBar

In [78]:
MapTrans

In [88]:
#simple maps of density centers
def CreateMap(df):
    # create map
    MyMap = folium.Map(location=[latitude, longitude], zoom_start=11)

    # adding markers to the map (R)
    markers_colors = []
    for lat, lng in zip(df['Lat'], df['Lng']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='0.75',
            fill=True,
            fill_color='0.75',
            fill_opacity=0.7).add_to(MyMap)
    
    return MyMap

In [92]:
MapRestC = CreateMap(RestC)
MapBarC = CreateMap(BarC)
MapTransC = CreateMap(TransC)

In [93]:
MapRestC

In [94]:
MapBarC

In [95]:
MapTransC

In [109]:
#simple maps of density centers for all data sets (R,B,T)
def CreateMapComb(df1, df2, df3):
    # create map
    MyMap = folium.Map(location=[latitude, longitude], zoom_start=11)

    # adding markers to the map (R)
    markers_colors = []
    for lat, lng in zip(df1['Lat'], df1['Lng']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='green',
            fill=True,
            fill_color='green',
            fill_opacity=0.7).add_to(MyMap)
        
    markers_colors = []
    for lat, lng in zip(df2['Lat'], df2['Lng']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.7).add_to(MyMap)
        
    markers_colors = []
    for lat, lng in zip(df3['Lat'], df3['Lng']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7).add_to(MyMap)
    
    return MyMap

In [117]:
CombinedCs = CreateMapComb(RestC, TransC, BarC)
CombinedCs

In [136]:
#maps of density centers for all data sets (R,B,T) with marked regions of interest
def CreateMapInterest(df1, df2, df3,df4):
    # create map
    MyMap = folium.Map(location=[latitude, longitude], zoom_start=11)

    # adding markers to the map (R)
    markers_colors = []
    for lat, lng in zip(df1['Lat'], df1['Lng']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='green', #positive
            fill=True,
            fill_color='green',
            fill_opacity=0.7).add_to(MyMap)
     
    # adding markers to the map (T)
    markers_colors = []
    for lat, lng in zip(df2['Lat'], df2['Lng']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue', #supporting
            fill=True,
            fill_color='blue',
            fill_opacity=0.7).add_to(MyMap)
        
    # adding markers to the map (B)
    markers_colors = []
    for lat, lng in zip(df3['Lat'], df3['Lng']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='red', #bad
            fill=True,
            fill_color='red',
            fill_opacity=0.7).add_to(MyMap)
        
    markers_colors = []
    for lat, lng in zip(df4['Lat'], df4['Lng']):
        folium.CircleMarker(
            [lat, lng],
            radius=30,
            color='yellow', #highlight
            fill=True,
            fill_color='yellow',
            fill_opacity=0).add_to(MyMap)
    
    return MyMap

In [146]:
#defining regions of interest
Highlight = pd.DataFrame(np.array([[47.41,8.55],[47.36,8.57],[47.35,8.52]]),
                         columns = ['Lat','Lng'])
Interests = CreateMapInterest(RestC, TransC, BarC,Highlight)
Interests